In [ ]:
from flask import Flask, request, jsonify
import mlflow
import mlflow.sklearn
import pandas as pd

# Configuración del servidor Flask
app = Flask(__name__)

# Configura el URI del servidor MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Cambia según tu servidor MLflow
model_name = "Diabetes"  # Nombre del modelo registrado
model_version = "3"  # Versión del modelo en MLflow

# Cargar el modelo desde el registro de MLflow
try:
    model_uri = f"models:/{model_name}/{model_version}"
    print(f"Cargando el modelo desde {model_uri}...")
    model = mlflow.sklearn.load_model(model_uri)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    model = None  # Manejar casos en los que el modelo no se carga correctamente

@app.route("/", methods=["GET"])
def home():
    """
    Endpoint para verificar que la API está funcionando.
    """
    return jsonify({"message": "El servidor Flask está funcionando correctamente."})

@app.route('/predict', methods=['POST'])
def predict():
    """
    Ruta para realizar predicciones usando el modelo registrado.
    """
    try:
        # Validar que el modelo esté cargado
        if model is None:
            return jsonify({'error': 'El modelo no está disponible. Revisa los logs para más información.'}), 500

        # Obtener datos enviados en formato JSON
        data = request.get_json()

        # Validar que los datos están presentes y tienen el formato correcto
        if not data:
            return jsonify({'error': 'No se recibieron datos.'}), 400
        if not isinstance(data, list):
            return jsonify({'error': 'Los datos deben enviarse como una lista de instancias.'}), 400

        # Convertir los datos en un DataFrame de Pandas
        input_data = pd.DataFrame(data)

        # Validar que el DataFrame no esté vacío
        if input_data.empty:
            return jsonify({'error': 'El conjunto de datos está vacío.'}), 400

        # Realizar la predicción
        predictions = model.predict(input_data)

        # Crear una respuesta JSON con las predicciones
        response = {
            'predictions': predictions.tolist(),
            'message': 'Predicciones realizadas con éxito.'
        }
        return jsonify(response)

    except Exception as e:
        # Manejo de errores
        return jsonify({'error': f"Error al realizar la predicción: {str(e)}"}), 500

if __name__ == '__main__':
    # Ejecutar la aplicación en el puerto 5001
    app.run(host='127.0.0.1', port=5001)
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5001, debug=True)


Cargando el modelo desde models:/Diabetes/3...
Modelo cargado exitosamente.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [16/Jan/2025 16:39:35] "GET / HTTP/1.1" 200 -
